# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [1]:
import re

def clean_split_text(text: str):
    text = re.sub(r'  ', ' ', text.lower())
    text = re.sub(r'[\\\,\.\-;!?]', '', text)
    return text.split()

def make_distance_matrix(ref_words: list, rec_words: list):
    distance_matrix = [[0] * (len(rec_words) + 1) for _ in range(len(ref_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(ref_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(rec_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(rec_words) + 1):
            prev_val = 0
            next_val = min(distance_matrix[i - 1][j - 1] + int(ref_words[i - 1] != rec_words[j - 1]), 
                           distance_matrix[i - 1][j] + 1, 
                           distance_matrix[i][j - 1] + 1)

            distance_matrix[i][j] = next_val
    
    return distance_matrix

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    
    reference_words = clean_split_text(reference_text)
    recognized_words = clean_split_text(recognized_text)

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = make_distance_matrix(reference_words, recognized_words)

    # Расчет WER  (в процентах)
    wer = distance_matrix[-1][-1] / len(reference_words) * 100
    return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [2]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [3]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
def get_next_min_distance(distance_matrix, ref_i, rec_i, is_s_accept = True):
    vals = []

    if (rec_i - 1 >= 0):
        vals.append(distance_matrix[rec_i - 1][ref_i])
    if (ref_i - 1 >= 0):
        vals.append(distance_matrix[rec_i][ref_i - 1])
    if (is_s_accept and rec_i - 1 >= 0 and ref_i - 1 >= 0):
        vals.append(distance_matrix[rec_i - 1][ref_i - 1])
        
    return min(vals)

def calulate_alignment(distance_matrix, reference_words, recognized_words):
    ali = [[], [], []]

    ref_i = len(reference_words)
    rec_i = len(recognized_words)
    for i in range(max(len(reference_words), len(recognized_words))):
        min_distance = get_next_min_distance(distance_matrix, rec_i, ref_i)

        if (min_distance == distance_matrix[ref_i - 1][rec_i - 1]):
            ref_i -= 1
            rec_i -= 1
            ali[0].append(reference_words[ref_i])
            ali[1].append(recognized_words[rec_i])
            ali[2].append('C' if reference_words[ref_i] == recognized_words[rec_i] else 'S')
        
        elif (min_distance == distance_matrix[ref_i][rec_i - 1]):
            rec_i -= 1
            ali[0].append('***')
            ali[1].append(recognized_words[rec_i])
            ali[2].append('I')
        elif (min_distance == distance_matrix[ref_i - 1][rec_i]):
            ref_i -= 1

            ali[0].append(reference_words[ref_i])
            ali[1].append('***')
            ali[2].append('D')
        else:
            assert(False)

    ali[0] = ali[0][::-1]
    ali[1] = ali[1][::-1]
    ali[2] = ali[2][::-1]

    return ali

def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    
    reference_words = clean_split_text(reference_text)
    recognized_words = clean_split_text(recognized_text)
    distance_matrix = make_distance_matrix(reference_words, recognized_words)

    # Перенесите сюда код из задания 1.a.
    wer = distance_matrix[-1][-1] / len(reference_words) * 100

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER
    ali = calulate_alignment(distance_matrix, reference_words, recognized_words)

    correct = len([i for i in ali[2] if i == 'C'])
    deletion = len([i for i in ali[2] if i == 'D'])
    insertion = len([i for i in ali[2] if i == 'I'])
    substitution = len([i for i in ali[2] if i == 'S'])

    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    return {"wer" : wer,
            "cor": correct, 
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": ali}


In [4]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно PunctuaionErrorRate (PER) по формуле

$$ PER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [127]:
def per_repl(per): 
    return ' ' + per[0] + ' '

def clean_split_text(text: str):
    text = re.sub(r'[\,\.\-;!?]', per_repl, text.lower())
    text = re.sub(r'  ', ' ', text)
    text = re.sub(r'[\\]', '', text)
    
    return text.split()

def is_punct(word):
    return word in ['.', ',', '!', '?', ':']

def make_distance_matrix(ref_words: list, rec_words: list):
    res = [[0] * (len(rec_words) + 1) for _ in range(len(ref_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(ref_words) + 1):
        res[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(rec_words) + 1):
        res[0][j] = j

    # print(res[0])
    # Заполнение матрицы расстояний методом динамического программирования
    for i, ref in enumerate(ref_words, start = 1):
        for j, rec in enumerate(rec_words, start = 1):
            
            ways = [res[i - 1][j] + 1, res[i][j - 1] + 1]
            if (is_punct(ref) == is_punct(rec)):
                ways.append(res[i - 1][j - 1] + int(ref != rec))

            res[i][j] = min(ways)

        # print(res[i])
    
    return res

def is_substitution_accept(reference_words, recognized_words, rec_i, ref_i):
    return (ref_i > 0) and (rec_i > 0) and (is_punct(reference_words[ref_i - 1]) == is_punct(recognized_words[rec_i - 1]))

def select_way(distance_matrix, reference_words, recognized_words, rec_i, ref_i):
    is_s_accept = is_substitution_accept(reference_words, recognized_words, rec_i, ref_i)
    min_distance = get_next_min_distance(distance_matrix, rec_i, ref_i, is_s_accept)

    ways_punct = []
    ways_word = []
    if is_s_accept and min_distance == distance_matrix[ref_i - 1][rec_i - 1]:
        if is_punct(reference_words[ref_i - 1]):
            ways_punct.append((ref_i - 1, rec_i - 1, 'S_p'))
        else:
            ways_word.append((ref_i - 1, rec_i - 1, 'S'))
    if min_distance == distance_matrix[ref_i][rec_i - 1]:
        if is_punct(recognized_words[rec_i - 1]):
            ways_punct.append((ref_i, rec_i - 1, 'I_p'))
        else:
            ways_word.append((ref_i, rec_i - 1, 'I'))
    if min_distance == distance_matrix[ref_i - 1][rec_i]:
        if is_punct(reference_words[ref_i - 1]):
            ways_punct.append((ref_i - 1, rec_i, 'D_p'))
        else:
            ways_word.append((ref_i - 1, rec_i, 'D'))

    if len(ways_punct) > 0:
        return ways_punct[0]
    
    return ways_word[0]

def calulate_alignment(distance_matrix, reference_words, recognized_words):
    ali = [[], [], []]

    ref_i = len(reference_words)
    rec_i = len(recognized_words)

    # print(ref_i, rec_i)
    while (ref_i > 0) or (rec_i > 0):
        ref_i, rec_i, label = select_way(distance_matrix, reference_words, recognized_words, rec_i, ref_i)

        if (label == 'S') or (label == 'S_p'):
            ali[0].append(reference_words[ref_i])
            ali[1].append(recognized_words[rec_i])
            label = 'C' if reference_words[ref_i] == recognized_words[rec_i] else label
        
        elif (label == 'I') or (label == 'I_p'):
            ali[0].append('***')
            ali[1].append(recognized_words[rec_i])

        elif (label == 'D') or (label == 'D_p'):
            ali[0].append(reference_words[ref_i])
            ali[1].append('***')

        else:
            assert(False)

        ali[2].append(label)
        # print(ref_i, rec_i)

    ali[0] = ali[0][::-1]
    ali[1] = ali[1][::-1]
    ali[2] = ali[2][::-1]

    return ali

def calculate_wer_per(reference_text: str, recognized_text: str):
    
    reference_words = clean_split_text(reference_text)
    recognized_words = clean_split_text(recognized_text)
    distance_matrix = make_distance_matrix(reference_words, recognized_words)

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER
    ali = calulate_alignment(distance_matrix, reference_words, recognized_words)

    word_cnt = len([w for w in reference_words if not is_punct(w)])
    punct_cnt = len(reference_words) - word_cnt

    wer = len([l for l in ali[2] if l in ['I', 'D', 'S']]) / word_cnt
    per = len([l for l in ali[2] if l in ['I_p', 'D_p', 'S_p']]) / punct_cnt
    
    return {"wer" : wer * 100,
            "per": per * 100,
            "ali": ali}

In [128]:
calculate_wer_per('привет студент.', 'привет. студент')

{'wer': 0.0,
 'per': 200.0,
 'ali': [['привет', '***', 'студент', '.'],
  ['привет', '.', 'студент', '***'],
  ['C', 'I_p', 'C', 'D_p']]}

In [129]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [ ]:
def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER
    sawer=0
    ali=[]
    return {"sawer" : sawer, 
            "ali": ali}


In [ ]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()